In [1]:
#listing dependencies
from bs4 import BeautifulSoup as bs
import pymongo
import requests
from flask import Flask, render_template
import pandas as pd
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
#setting up splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless = False)



====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Driver [C:\Users\Andrew\.wdm\drivers\chromedriver\win32\93.0.4577.15\chromedriver.exe] found in cache


# NASA Mars News

In [3]:
nasa_mars_news_url = 'https://redplanetscience.com/'
browser.visit(nasa_mars_news_url)
html = browser.html
soup = bs(html, 'html.parser')

In [4]:
#showing div with content needed to scrape
# div_container = soup.find_all('div', class_ = 'list_text')
# div_container

In [5]:
#putting date, title, and p into variables
latest_date = soup.find_all('div', class_ = 'list_date')[0].text
news_title = soup.find_all('div', class_ = 'content_title')[0].text
news_p = soup.find_all('div', class_ = 'article_teaser_body')[0].text
print(latest_date)
print(news_title)
print(news_p)

September 2, 2021
How NASA's Mars Helicopter Will Reach the Red Planet's Surface
The small craft will seek to prove that powered, controlled flight is possible on another planet. But just getting it onto the surface of Mars will take a whole lot of ingenuity.


# JPL Mars Space Images - Featured Image

In [6]:
jpl_url = 'https://spaceimages-mars.com/'
browser.visit(jpl_url)

In [7]:
html = browser.html
soup = bs(html, 'html.parser')
feature_image = soup.find_all('img', class_ = 'headerimage')[0]['src']
# feature_image

In [8]:
featured_image_url = jpl_url + feature_image
featured_image_url

'https://spaceimages-mars.com/image/featured/mars1.jpg'

# Mars Facts

In [9]:
facts_url = 'https://galaxyfacts-mars.com/'
tables = pd.read_html(facts_url)
#I had to install lxml for this to run successfully

In [10]:
mars_table = tables[1]
mars_table  =mars_table.rename(columns = {0:"Label", 1:"Value"})
mars_table = mars_table.set_index("Label")
mars_table

,Value
Label,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 ( Phobos & Deimos )
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [11]:
mars_table_html = mars_table.to_html()
print(mars_table_html)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Value</th>
    </tr>
    <tr>
      <th>Label</th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>Equatorial Diameter:</th>
      <td>6,792 km</td>
    </tr>
    <tr>
      <th>Polar Diameter:</th>
      <td>6,752 km</td>
    </tr>
    <tr>
      <th>Mass:</th>
      <td>6.39 × 10^23 kg (0.11 Earths)</td>
    </tr>
    <tr>
      <th>Moons:</th>
      <td>2 ( Phobos &amp; Deimos )</td>
    </tr>
    <tr>
      <th>Orbit Distance:</th>
      <td>227,943,824 km (1.38 AU)</td>
    </tr>
    <tr>
      <th>Orbit Period:</th>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <th>Surface Temperature:</th>
      <td>-87 to -5 °C</td>
    </tr>
    <tr>
      <th>First Record:</th>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <th>Recorded By:</th>
      <td>Egyptian astronomers</td>
    </tr>
  </tbody>
</table>


# Mars Hemispheres

In [12]:
hemi_url = 'https://marshemispheres.com/'
browser.visit(hemi_url)
html  = browser.html
soup = bs(html, 'html.parser')

In [13]:
#trying to get links to each hemi
div_item = soup.find_all('a', class_ = 'itemLink')
# div_item
hemi_list = []
for x in div_item:
    hemi_html = x['href']
    hemi_list.append(hemi_html)

In [14]:
#cleaning list of hemi htmls
hemi_list = list(set(hemi_list))
hemi_list = [y for y in hemi_list if ".html" in y]

In [15]:
hemisphere_image_urls = []
for hemi in hemi_list:
    browser.visit(hemi_url + hemi)
    html = browser.html
    soup = bs(html, 'html.parser')
    title = soup.find('h2', class_ = 'title').text
    hemi_img_li = soup.find_all('li')[0]
    hemi_img = hemi_img_li.find('a')['href']
    img_url = hemi_url + hemi_img
    hemi_dict = {
        "title": title,
        "img_url": img_url
    }
    hemisphere_image_urls.append(hemi_dict)

In [16]:
hemisphere_image_urls

[{'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg'},
 {'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg'}]

In [ ]:
# #testing for loop code
# browser.visit(hemi_url + hemi_list[0])
# html = browser.html
# soup = bs(html, 'html.parser')
# title = soup.find('h2', class_ = 'title').text
# # title
# hemi_img = soup.find_all('li')[0]
# test = hemi_img.find('a')['href']
# test